In [2]:
# %pip uninstall 

In [3]:
# import tensorflow as tf
# print(tf.config.list_physical_devices('GPU'))

In [4]:
# import numpy as np
# import tensorflow as tf
# Sequential= tf.keras.models.Sequential
# Embedding= tf.keras.layers.Embedding
# Conv2D= tf.keras.layers.Conv2D
# MaxPooling2D= tf.keras.layers.MaxPooling2D
# GlobalAveragePooling2D= tf.keras.layers.GlobalAveragePooling2D
# Droput= tf.keras.layers.Dropout
# Flatten= tf.keras.layers.Flatten
# Dense= tf.keras.layers.Dense
# Activation= tf.keras.layers.Activation


In [ ]:

# from tensorflow import keras
# from tensorflow.python.keras import backend as K
# from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras import layers
# from tensorflow.keras import activations
# from tensorflow.keras import optimizers
# from tensorflow.keras import losses
# from tensorflow.keras import initializers
# from tensorflow.keras import regularizers
# from tensorflow.keras.utils import to_categorical, plot_model
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
# # Import pretrained models
# from tensorflow.keras.applications import Xception

# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report

In [6]:
# import tensorflow as tf
# AUTOTUNE = tf.data.experimental.AUTOTUNE

In [7]:
from tensorflow.keras.layers import Input, Conv2D, SeparableConv2D, Add, Dense, BatchNormalization, ReLU, MaxPool2D, GlobalAvgPool2D

def conv_bn(x, filters, kernel_size, strides=1):
    x= Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same', use_bias=False)(x)
    x= BatchNormalization()(x)
    return x

def sep_bn(x, filters, kernel_size, strides=1):
    x= SeparableConv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same', use_bias=False)(x)
    x= BatchNormalization()(x)
    return x

Entry Flow

In [8]:
def entry_flow(x):
    x= conv_bn(x, filters=32, kernel_size=3, strides=2)
    x= ReLU()(x)
    x= conv_bn(x, filters=64, kernel_size=3)
    tensor= ReLU()(x)
    
    x=sep_bn(tensor, filters=128, kernel_size=3)
    x= ReLU()(x)
    x=sep_bn(x, filters=128, kernel_size=3)
    x= MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    
    tensor= conv_bn(tensor, filters=128, kernel_size=1, strides=2)
    x= Add()([tensor, x])
    x= ReLU()(x)
    x=sep_bn(x, filters=256, kernel_size=3)
    x= ReLU()(x)
    x=sep_bn(x, filters=256, kernel_size=3)
    x= MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    tensor= conv_bn(tensor, filters=256, kernel_size=1, strides=2)
    x= Add()([tensor, x])
    x= ReLU()(x)
    x=sep_bn(x, filters=728, kernel_size=3)
    x= ReLU()(x)
    x=sep_bn(x, filters=728, kernel_size=3)
    x= MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    tensor= conv_bn(tensor, filters=728, kernel_size=1, strides=2)
    x= Add()([tensor, x])
    
    return x

Middle Flow

In [9]:
def middle_flow(tensor):
    for _ in range(8):
        x= ReLU()(tensor)
        x=sep_bn(x, filters=728, kernel_size=3)
        x= ReLU()(x)
        x=sep_bn(x, filters=728, kernel_size=3)
        x= ReLU()(x)
        x=sep_bn(x, filters=728, kernel_size=3)
        tensor= Add()([tensor, x])
    return tensor
        

Exit Flow

In [10]:
def exit_flow(tensor):
    x=ReLU()(tensor)
    x=sep_bn(x, filters=728, kernel_size=3)
    x= ReLU()(x)
    x=sep_bn(x, filters=1024, kernel_size=3)
    x= MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    tensor= conv_bn(tensor, filters=1024, kernel_size=1, strides=2)
    x= Add()([tensor, x])
    x=sep_bn(x, filters=1536, kernel_size=3)
    x= ReLU()(x)
    x=sep_bn(x, filters=2048, kernel_size=3)
    x= ReLU()(x)
    x= GlobalAvgPool2D()(x)
    x= Dense(units= 1000, activation='softmax')(x)
    return x
    

Model Code

In [11]:
input= Input(shape=(299, 299, 3))
x= entry_flow(input)
x= middle_flow(x)
output= exit_flow(x)

from tensorflow.keras import Model
model= Model(inputs= input, outputs= output)

In [12]:
from tensorflow.keras.utils import plot_model
plot_model(model, show_shapes=True)

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.771951 to fit



In [15]:
import numpy as np
import tensorflow.keras.backend as K
np.sum([K.count_params(p) for p in model.trainable_weights])

np.int64(22855952)